# Data Augmentation

In [ ]:
# Import necessary libraries
import cv2 as cv
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, save_img, img_to_array, load_img
import os 
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [ ]:
# Check for available GPUs
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available:", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0],True)

In [ ]:
# function to save frames from videos
def getFrames(video_path, desired_dir):
    
    cap = cv.VideoCapture(video_path)
    i = 0
    frameCount = 0
    while (True):
        isTrue, frame = cap.read()
        if frameCount%10 == 0:
            if not isTrue:
                break
            
            #-------------------------------------------------------------------------------------------------------------------------------
            # Saving every 10th frame from labelled videos (train,validation and test data)
            #-------------------------------------------------------------------------------------------------------------------------------
            name = desired_dir+"/frame"+str(i)+".jpg"

            cv.imwrite(name, frame)
            i += 1
        frameCount += 1


In [ ]:
# Organize data into train, validation, test directories
os.chdir('train_data')
if os.path.isdir('train/Fissure') is False:
    os.makedirs('train/Fissure')
    os.makedirs('train/Racines')
    os.makedirs('train/Normal')

    os.makedirs('valid/Fissure')
    os.makedirs('valid/Racines')
    os.makedirs('valid/Normal')
    
    os.makedirs('test/Fissure')
    os.makedirs('test/Racines')
    os.makedirs('test/Normal')

    for i in random.sample(glob.glob('Fissure*'), 358):
        shutil.move(i, 'train/Fissure')      
    for i in random.sample(glob.glob('Racines*'), 122):
        shutil.move(i, 'train/Racines')
    for i in random.sample(glob.glob('Normal*'), 363):
        shutil.move(i, 'train/Normal')      
    
    for i in random.sample(glob.glob('Fissure*'), 200):
        shutil.move(i, 'test/Fissure')      
    for i in random.sample(glob.glob('Racines*'), 100):
        shutil.move(i, 'test/Racines')
    for i in random.sample(glob.glob('Normal*'), 300):
        shutil.move(i, 'test/Normal')
    
    for i in random.sample(glob.glob('Fissure*'), 300):
        shutil.move(i, 'valid/Fissure')      
    for i in random.sample(glob.glob('Racines*'), 100):
        shutil.move(i, 'valid/Racines')
    for i in random.sample(glob.glob('Normal*'), 300):
        shutil.move(i, 'valid/Normal') 

os.chdir('../../')

In [ ]:
# Define train driectory, validation directory and test directory
train_path = 'D:/IST/5 ano/1 Semestre/Machine Learning/Project/Part 2/Proj/train_data/train'
valid_path = 'D:/IST/5 ano/1 Semestre/Machine Learning/Project/Part 2/Proj/train_data/valid'
test_path = 'D:/IST/5 ano/1 Semestre/Machine Learning/Project/Part 2/Proj/train_data/test'

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# Perform data augmentation on the training data set (not on the validation nor on the test data sets)
#----------------------------------------------------------------------------------------------------------------

# Define directory
os.chdir(train_path)
if os.path.isdir('aug') is False:
    os.makedirs('aug')

labels = ['Fissure', 'Racines', 'Normal']

# Change the brightness of the image
brightness = ImageDataGenerator(brightness_range=[0.2,1.8])

# Apply vertical and horizontal flip
randomFlip = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),])

# Apply random rotation of the image
randomRot = tf.keras.Sequential([layers.RandomRotation(0.3, fill_mode='reflect'),])

# Change contrast of the image
randomCont = tf.keras.Sequential([layers.RandomContrast(0.4),])

# Apply vertical translation with periodic boundaries
randomTransVert = tf.keras.Sequential([layers.RandomTranslation(0.3, 0, fill_mode='wrap'),])

# Apply horizontal translation with periodic boundaries
randomTransHorz = tf.keras.Sequential([layers.RandomTranslation(0, 0.3, fill_mode='wrap'),])

# Apply a combination of vertical/horizontal flip with rotation
randomFlipRot = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.3, fill_mode='reflect'),])

# Apply a combination of contrast with rotation
randomContRot = tf.keras.Sequential([layers.RandomContrast(0.4),layers.RandomRotation(0.3, fill_mode='reflect'),])

#---------------------------------------------------------------------------------------------------------------------
# Apply brightness change for every image in the directory for each of the labels
#---------------------------------------------------------------------------------------------------------------------
for label in labels:
    img_num = 0
    for pic in os.listdir(label):
        path = train_path+'/'+label
        os.chdir(path)

        img = load_img(pic)
        img = img_to_array(img)
        img = np.expand_dims(img[10:576-10,10:788-20,:], axis=0)
        
        i = 0
        iter_brightness = brightness.flow(img, batch_size = 1)
        while i < 10:
            batch_brightness = iter_brightness.next()
            image = batch_brightness[0].astype('uint8')
            name = train_path+'/aug/'+label+'B'+str(img_num)+str(i)+'.jpg'
            save_img(name, image)

            i += 1
        
        img_num += 1
    
    os.chdir(train_path)


#---------------------------------------------------------------------------------------------------------------------
# Apply the other methods for every image in the directory for each of the labels
#---------------------------------------------------------------------------------------------------------------------
for label in labels:
    img_num = 0
    for pic in os.listdir(label):
        path = train_path+'/'+label
        os.chdir(path)

        img = load_img(pic)
        data_tensor = tf.convert_to_tensor(img)
        image = tf.expand_dims(data_tensor[10:576-10,10:788-20,:], 0)
        
        i = 0
        while i < 10:
            flip = randomFlip(image)
            proto_tensor = tf.make_tensor_proto(flip[0])
            img = tf.make_ndarray(proto_tensor)
            name = train_path+'/aug/'+label+'Flip'+str(img_num)+str(i)+'.jpg'
            save_img(name, img)

            rot = randomRot(image)
            proto_tensor = tf.make_tensor_proto(rot[0])
            img = tf.make_ndarray(proto_tensor)
            name = train_path+'/aug/'+label+'Rot'+str(img_num)+str(i)+'.jpg'
            save_img(name, img)

            cont = randomCont(image)
            proto_tensor = tf.make_tensor_proto(cont[0])
            img = tf.make_ndarray(proto_tensor)
            name = train_path+'/aug/'+label+'Cont'+str(img_num)+str(i)+'.jpg'
            save_img(name, img)

            transV = randomTransVert(image)
            proto_tensor = tf.make_tensor_proto(transV[0])
            img = tf.make_ndarray(proto_tensor)
            name = train_path+'/aug/'+label+'TransV'+str(img_num)+str(i)+'.jpg'
            save_img(name, img)
            
            transH = randomTransHorz(image)
            proto_tensor = tf.make_tensor_proto(transH[0])
            img = tf.make_ndarray(proto_tensor)
            name = train_path+'/aug/'+label+'TransH'+str(img_num)+str(i)+'.jpg'
            save_img(name, img)

            flipRot = randomFlipRot(image)
            proto_tensor = tf.make_tensor_proto(flipRot[0])
            img = tf.make_ndarray(proto_tensor)
            name = train_path+'/aug/'+label+'FlipRot'+str(img_num)+str(i)+'.jpg'
            save_img(name, img)

            contRot = randomContRot(image)
            proto_tensor = tf.make_tensor_proto(contRot[0])
            img = tf.make_ndarray(proto_tensor)
            name = train_path+'/aug/'+label+'ContRot'+str(img_num)+str(i)+'.jpg'
            save_img(name, img)

            i += 1
        
        img_num += 1
    
    os.chdir(train_path)




In [ ]:
# Organize augmented data into respective class
pathAug = train_path+'/aug'
pathFissure = train_path+'/Fissure'
pathRacines = train_path+'/Racines'
pathNormal = train_path+'/Normal'

os.chdir(pathAug)

for i in random.sample(glob.glob('Fissure*'), 28640):
    shutil.move(i, pathFissure)      
for i in random.sample(glob.glob('Racines*'), 9760):
    shutil.move(i, pathRacines)
for i in random.sample(glob.glob('Normal*'), 29040):
    shutil.move(i, pathNormal)
